# IRTM programming assignment 2

In [39]:
import nltk
from nltk.stem import PorterStemmer
import math
import numpy as np

In [40]:
documents = []
doc_term_freqs = []
df = {}

stop_words = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
    'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
    'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 
    'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 
    'should', 'now'
]

stemmer = PorterStemmer()

In [41]:
def remove_non_letters(content):
    return ''.join([char if char.isalpha() else ' ' for char in content])

In [42]:
for i in range(1, 1096):
    filename = f'./data/{i}.txt'
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
        
    content = remove_non_letters(content)
        
    tokens = content.split()
    
    tokens = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    
    documents.append(tokens)
    
    term_freq = {}
    for token in tokens:
        term_freq[token] = term_freq.get(token, 0) + 1
    doc_term_freqs.append(term_freq)
    
    unique_tokens = set(tokens)
    for token in unique_tokens:
        df[token] = df.get(token, 0) + 1

In [43]:
N = len(documents)

idf = {}
for term, freq in df.items():
    idf[term] = math.log10(N / freq)

vocabulary = sorted(idf.keys())
term_index = {term: idx for idx, term in enumerate(vocabulary)}

tf_idf_matrix = np.zeros((N, len(vocabulary)))

for doc_idx, term_freq in enumerate(doc_term_freqs):
    for term, freq in term_freq.items():
        if term in term_index:
            idx = term_index[term]
            tf_idf_matrix[doc_idx, idx] = freq * idf[term]

tf_idf_norm = np.linalg.norm(tf_idf_matrix, axis=1, keepdims=True)
tf_idf_norm[tf_idf_norm == 0] = 1
tf_idf_matrix = tf_idf_matrix / tf_idf_norm

In [44]:
with open('./output/dictionary.txt', 'w', encoding='utf-8') as file:
    file.write('t_index\tterm\tdf\n')
    for index, term in enumerate(vocabulary, start=1):
        file.write(f'{index}\t{term}\t{df[term]}\n')

In [45]:
document_number = 1
vector = tf_idf_matrix[document_number - 1]
output_filename = f'./output/{document_number}.txt'
with open(output_filename, 'w', encoding='utf-8') as f:
    for idx, value in enumerate(vector, start=1):
        if value != 0:
            f.write(f'{idx}\t{value:.6f}\n')

In [48]:
def cosine(Docx, Docy):
    Docx_index = Docx - 1
    Docy_index = Docy - 1
    
    vector_x = tf_idf_matrix[Docx_index]
    vector_y = tf_idf_matrix[Docy_index]
    
    return np.dot(vector_x, vector_y)

In [52]:
similarity = cosine(1, 1095)
print(similarity)

0.048482080659025026
